# LAB 02: Cryptocurrency Daily Market Price

## Thông tin người thực hiện
|  MSSV |Họ và tên   |
|---|---|
|  20120095 |Ngô Quang Hưng   |
|  20120519|Nguyễn Thị Thúy Liễu   |
|  20120540|Võ Hoàng Thảo Nguyên   |
|  20120633|Viên Hải Yến   |
|  20120634|Lê Minh Trí   |


## Mô tả đồ án

Import các thư viện

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from datetime import datetime, timedelta
import warnings

### <span style="color:yellow"> A. Thu thập dữ liệu</span>

#### Ngữ cảnh thực hiện tìm kiếm dữ liệu

Đây là thời đại công nghê số nơi mà thị trường điện tử phát triển mạnh mẽ và được nhiều người biết đến. Dataset này cung cấp thông tin chi tiết về lịch sử giao dịch của các loại tiền điện tử, bao gồm các chỉ số quan trọng như giá mở, giá cao, giá thấp, giá đóng cửa, khối lượng giao dịch và vốn hóa thị trường, ...\
Bộ dữ liệu lưu trữ trong khoảng thời gian từ năm 2013-2018 về nhiều loại tiền ảo ở các thời gian gần nhau để thấy được sự chênh lệch trong các khoảng thời gian xác định.

#### Chủ đề

Time-series, phân tích độ trending, rủi ro, tầm phát triển, thay đổi của thị trường Crypo từ năm 2013-2018

#### Nguồn

Dataset trên Kaggle: [Tại đây](https://www.kaggle.com/datasets/jessevent/all-crypto-currencies?fbclid=IwAR1ClYMb9dVjpBwOPuqJfWkRf5gLsukrWscsuV_okKL4X6Q9e7JkjcGtdQ8)

#### Người ta có cho phép sử dụng dữ liệu như thế này hay không?

Theo thông tin License của dữ liệu, người dùng được phép tự do chia sẻ, sửa đổi và sử dụng dữ liệu trong khi duy trì sự tự do tương tự cho người dùng khác.

#### Người ta đã thu thập dữ liệu này như thế nào? Phương pháp thực hiện là gì?

Phương pháp thu thập dữ liệu:
- Thu thập thông tin bằng code và sử dụng CRAN của R (lưu trữ trực tuyến thường được sử dụng với R packages và các tài nguyên R khác).
- Do đó, có những cải tiến đáng kể về cách một số chuyển đổi trường đã được thực hiện và dữ liệu được làm sạch.

#### Đọc dữ liệu

In [ ]:
# dateparse = lambda dates: pd.datetime.strptime(dates, '%Y-%m-%d')
df = pd.read_csv('./dataset/crypto-markets.csv', parse_dates=['date'], index_col='date')
df.head()

----

### <span style="color:yellow"> B. Khám phá dữ liệu</span>

#### Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

- Mỗi dòng chứa dữ liệu về một loại tiền ảo và các thông tin liên quan trong một thời điểm cụ thể. Thông tin bao gồm: thời gian công bố, tên loại tiền, tên viết tắt, xếp hạng, giá mở cửa, giá đóng cửa, giá cao nahastm giá thấp nhất, khối lượng giao dịch, vốn hoá thị trường, chênh lệch giữa giá mua và giá bán ,tỉ lệ đóng.

- ví dụ dòng 1: tiền bitcoin có xếp hạng là 1, giá mở là 135.3, giá đóng là 134.21, giá cao nhất là 135.98, giá thấp nhất là 132.10, khối lượng giao dịch là 0, vốn hoá thị trường là 1.488567e+09, tỉ lệ đóng 0.5438, chênh lệch giá mua và giá bán là 3.88.

- Không có tình trạng các dòng có ý nghĩa khác nhau.


#### Mỗi cột có ý nghĩa gì?

In [ ]:
column=11
print("column ", column,": ", df.columns[column])
print("size: ", len(df.loc[:, df.columns[column]]))
print("number of values: ", len(df.loc[:, df.columns[column]].unique()))

arr= df.loc[:, df.columns[column]].unique()

nanIndex=-1
for index in range(len(arr)):
    if isinstance(arr[index], float) and np.isnan(arr[index]):
        nanIndex=index
        break
if nanIndex != -1:
    arr=np.delete(arr,nanIndex)

print("number of values (without nan): ", len(arr))
print("data (without nan) : ", arr[0])
print("data (without nan) : ", arr[1])
print("data (without nan) : ", arr[2])
print("data (without nan) : ", arr[3])

print(min(arr))
print(max(arr))
print(np.mean(arr))
# print("data (without nan) : ", ', '.join(arr))

#### Thông tin mỗi cột

- Slug: Tên loại tiền
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 2071
    - Một số dữ liệu: bitcoin, ripple, ethereum, stellar, bitcoin-cash,...
- symbol: tên viết tắt của loại tiền
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 2005
    - Một số dữ liệu:  BTC, XRP, ETH, XLM, BCH, EOS, LTC, USDT, BSV, ADA,...
- name: tên loại tiền
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 2071
    - Một số dữ liệu:  Bitcoin, XRP, Ethereum, Stellar, Bitcoin Cash, EOS,...
- ranknow: xếp hạng
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 2068
    - Một số dữ liệu:  1,2,3,...,2070,2071,2072.
- open: giá mở
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 266140
    - Một số dữ liệu:  135.3, 134.44, 144.,...,23320.9, 24543.53, 22872.09
    - min: 2.5e-09
    - max: 2298390.0
    - Trung bình: 1222.88
- high: gía cao nhất
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 269676
    - Một số dữ liệu:  135.98, 147.49, 146.93, ... ,25235.1,  24582.47, 23086.98
    - min: 3.2e-09
    - max: 2926100.0
    - Trung bình: 1417.51
- low: gía thấp nhất
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 263008
    - Một số dữ liệu:  132.1, 134., 134.05, ... ,19888.1,  21457.95, 19040.91
    - min: 2.5e-10
    - max: 2030590.0
    - Trung bình: 1052.11
- close: gía đóng
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 265887
    - Một số dữ liệu:  134.21, 144.54, 139., ... ,24442.7,  22839.44, 21136.72
    - min: 2e-10
    - max: 2300740.0
    - Trung bình: 1215.35
- volume: khối lượng giao dịch
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 300528
    - Một số dữ liệu:  0, 46862700.0, 32505800.0, 19011300.0,...
    - min: 0.0
    - max: 23840899072.0
    - Trung bình: 27175312.58
- market: vốn hoá thị trường
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 660996
    - Một số dữ liệu:  1488566728.0, 1603768865.0, 1542813125.0, 1298954594.0,...
    - min: 0.0
    - max: 326502485530.0
    - Trung bình: 245833831.28
- spread: chênh lệch giá mua và giá bán
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 12454
    - Một số dữ liệu:  3.88, 13.49, 12.88, 32.17,...
    - min: 0
    - max: 1770563.0
    - Trung bình: 8472.75
- close_ratio: tỉ lệ đóng
    - Kích thước: 942297
    - Số lượng dữ liệu không trùng lắp: 10022
    - Một số dữ liệu:  0.5438, 0.7813, 0.3843, 0.2882,...
    - min: -1.0
    - max: 1
    - Trung bình: 0.49

#### Mỗi cột hiện đang có kiểu dữ liệu gì?

In [ ]:
df.dtypes

In [ ]:
object_columns = df.select_dtypes(include=['object']).columns
for col in object_columns:
    print(col,":",set(df[col].apply(type)))

#### Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

Các cột đã ở kiểu dữ liệu phù hợp để phân tích

#### Phân bố dữ liệu ở các cột

##### Với mỗi cột có kiểu dữ liệu dạng numeric, các giá trị được phân bố như thế nào?

In [ ]:
numerical_des = df.describe()
missing_ratio = pd.DataFrame({col : df[col].isnull().sum() / len(df) for col in numerical_des.columns}, index=["missing_ratio"])
pd.concat([numerical_des,missing_ratio])

##### Với mỗi cột không có kiểu dữ liệu dạng categorical, các giá trị được phân bố như thế nào?

In [ ]:
cate_rows=["missing_ratio","nums_of_vals", "num_diff_vals","diff_vals"]
data_cate={}

for col in df.columns:
    if df[col].dtype == 'object':
        row=[]
        row.append(round(df[col].isnull().sum()/len(df[col])*100,3))
        data_cate.update({f'{col}':row})

        row.append(len(df[col].notna()))
        row.append(len(df[col][df[col].notna()].unique()))
        row.append(np.array(df[col][df[col].notna()].unique()))
        data_cate.update({f'{col}':row})

cate_col_df=pd.DataFrame(data_cate,index=cate_rows)
display(cate_col_df)

#### Tiền xử lí dữ liệu

In [ ]:

print(df.info())
print(df.describe())
# kiểm tra missing data
print(df.isna().sum())

# Kiểm tra có dữ liệu bị duplicate không
print(str(df.duplicated().sum()))

# Kiểm ra khoản thời gian của dữ liệu
print(df.index.min())
print(df.index.max())

- 0 Nan
- 1380 duplicated
- Dữ liệu thu thập được từ 2013-04-28 đến 2018-11-30

Dữ liệu tương đối sạch ta chỉ cần phải tiền xử lý dữ liệu thêm 1 chút.

In [ ]:
# xữ lý dữ liệu duplicate và missing
df.drop_duplicates(inplace=True)
df.fillna(method='ffill', inplace=True)

# chọn lấy dữ liệu từ 2014-01-01 đến 2018-01-01
df=df.loc['2014-01-01':'2018-01-01']
print(df.head(),df.tail())

### <span style="color:yellow"> C. Khám phá mối quan hệ trong dữ liệu</span>